# Tree data structure

The abstract class definition of tree. 

In [12]:
import jdc
class Tree:
    class Position:
        def element(self):
            raise NotImplementedError('must be implemented by subclass')
        
        def __eq__(self, other):
            raise NotImplementedError('must be implemented by subclass')
            
        def __ne__(self, other):
            return not(self == other)
        
    def root(self):
        raise NotImplementedError('must be implemented by subclass')
        
    def parent(self):
        raise NotImplementedError('must be implemented by subclass')
        
    def num_children(self, p):
        raise NotImplementedError('must be implemented by subclass')
        
    def children(self, p):
        raise NotImplementedError('must be implemented by subclass')
        
    def __len__(self):
        raise NotImplementedError('must be implemented by subclass')
        
    def is_root(self, p):
        return self.root() == p
    
    def is_leaf(self, p):
        return self.num_children(p) == 0
    
    def is_empty(self):
        return len(self) == 0
    
    def depth(self, p):
        if self.is_root(p):
            return 0
        else:
            return 1 + self.depth(self.parent(p))
    
    def _height(self, p):
        if self.is_leaf():
            return 0
        else:
            return 1 + max(self._height(c) for c in self.children(p))
    
    def height(self, p = None):
        if p is None:
            p = self.root()
        return self._height(p)
    
        

## Recursive binary tree definition

Incidentally, we can also define a binary tree in a recursive way such that binary tree is either empty or consits of:
    * A node r, called the root of T, that stores an element
    * A binary tree (possibly empty), called the left subtree of T
    * A binary tree (possibly empty), called the right subtree of T

#### a binary tree is a specialization of a tree that supports three additional accessor methods:
    * T.left(p):   return the left child of node p or None if p has no left child
    * T.right(p):  return the right child of node p or None if p has no right child
    * T.sibling(p):return the position that represents the sibling of p or None if p has no sibling.

In [13]:
class BinaryTree(Tree):
    """Abstract base class that represents binary tree"""
    
    def left(self, p):
        """Return a Position representing p's left child"""
        raise NotImplementedError('must be implemented in subclass')
        
    def right(self, p):
        """Return a Position representing p's right child"""
        raise NotImplementedError('must be implemented in subclass')
        
    def sibling(self, p):
        """Return a Position representing the p's sibling or None if no sibling"""
        parent = self.parent(p)
        if parent is None:
            return None
        else:
            if p == self.left(parent):
                return self.right(parent)
            else:
                return self.left(parent)
    
    def children(self, p):
        """Generate an iteration of Positions representing p's children"""
        if self.left(p) is not None:
            yield self.left(p)
        if self.right(p) is not None:
            yield self.right(p)
    
        

## Linked structure for Binary Tree

For linked binary trees, a resonable set of update methods to support for general usage are the flowwing:
    * T.add_root(e): create the root of an empty tree, storing e as the  element and return the position of that root; an error occurs if the tree is not empty.
    * T.add_left(p, e): create a new node storing element e, link the node as the left child of position p, and return the resulting position; an error occurs if p already has a left child.
    * T.add_right(p, e): same as above
    * T.replace(p, e): replace the element stored at postion p with element e, and return the previously stored element.
    * T.delete(p): remove the node at position p, replacing it with its child, if any, and return the element that had been stored at p; an error occurs if p has two children.
    * T.attach(p, T1, T2): attach the internal structure of tree T1 and T2, respectively, as the left and right subtrees of leaf position p of T, and reset T1, and T2 to empty trees; an error condition occurs if p is not a leaf.
    
All the operations can be implemented in O(1) worst-case time with our linked representation. 

In [14]:
class LinkedBinaryTree(BinaryTree):
    """Linked representation of a binary tree structure"""
    
    class _Node:
        """Lightweight, nonpublic class for storing a node"""
    
        __slots__ = '_element', '_parent', '_left', '_right'
    
        def __init__(self, element, parent = None, left = None, right = None):
            self._element = element
            self._parent = parent
            self._left = left
            self._right = right
        
    
    class Position(BinaryTree.Position):
        """An abstraction representing the location of a single element"""
        
        def __init__(self, container, node):
            self._container = container
            self._node = node
            
        def element(self):
            return self._node._element
        
        def __eq__(self, other):
            return type(other) is type(self) and other._node == self._node
        
    def _validate(self, p):
        """Return associated node, if position is valid"""
        if not isinstance(p, self.Position):
            raise TypeError('p must be proper Position type')
        if p._container is not self:
            raise ValueError('p does not belong to this container')
        if p._node._parent is p._node:
            raise ValueError('p is no longer valid')
        return p._node
    
    def _make_position(self, node):
        """Return Position instance for given node (or None if no node)"""
        return self.Position(self, node) if node is not None else None
    
    def __init__(self):
        self._root = None
        self._size = 0
        
    def __len__(self):
        return self._size
    
    def root(self):
        return self._make_position(self._root)
    
    def parent(self, p):
        node = self._validate(p)
        return self._make_position(node._parent)
    
    def left(self, p):
        node = self._validate(p)
        return self._make_position(node._left)
    
    def right(self, p):
        node = self._validate(p)
        return self._make_position(node._right)
    
    def num_children(self, p):
        node = self._validate(p)
        count = 0
        if node._left is not None:
            cout += 1
        if node._right is not None:
            cout += 1
        return count
    
    def _add_root(self, e):
        if self._root is not None:
            raise ValueError('Root exists')
        self._size = 1
        self._root = self._Node(e)
        return self._make_position(self._root)
    
    def _add_left(self, p, e):
        """Create a new left child for Position p, storing element e
        Return the Position of new node
        Raise ValueError if Position p is invalid or p already has a left child
        """
        node = self._validate(p)
        if node._left is not None:
            raise ValueError('Left child exists')
        self._size += 1
        node._left = self._Node(e, node)
        return self._make_position(node._left)
    
    def _add_right(self, p, e):
        """Create a new right child for Position p, storing element e
        Return the Position of new node
        Raise ValueError if Position p is invalid or p already has a right child
        """
        node = self._validate(p)
        self._size += 1
        node._right = self._Node(e, node)
        return self._make_position(node._right)
    
    def _replace(self, p, e):
        """Replace the element at position p with e, and return old element"""
        node = self._validate(p)
        old = node._element
        node._element = e
        return old
    
    def _delete(self, p):
        """Delete the node at Position p, and replace it with its child, if any
        Return the element that had been stored at Position p
        Raise ValueError if Position p is invalid or p has two children
        """
        node = self._validate(p)
        if self.num_children(p) == 2:
            raise ValueError('p has two children')
        child = node._left if node._left else node._right
        
        if child is not None:
            child._parent = node._parent
        if node is self._root:
            self._root = child
        else:
            parent = node._parent
            if node is parent._left:
                parent._left = child
            else:
                parent._right = child
        self._size -= 1
        node._parent = node
        return node._element
    
    def _attach(self, p, t1, t2):
        """Attach trees t1 and t2 as left and right subtrees of external p"""
        node = self._validate(p)
        if not self.is_leaf(p):
            raise ValueError('position must be leaf')
            
        if not type(self) is type(t1) is type(t2):
            raise TypeError('Tree types must match')
            
        self._size += len(t1) + len(t2)
        
        if not t1.is_empty():
            t1._root._parent = node
            node._left = t1._root
            t1._root = None
            t1._size = 0
        if not t2.is_empty():
            t2._root._parent  = node
            node._right = t2._root
            t2._root = None
            t2._size = 0

In summary the performance of the linked structure implementation of a binary tree

|Operation|Running time|
|---------|------------|
|len, is_empty| O(1)|
|root, parent, left, right, sibling, children, num_children|O(1)|
|is_root, is_leaf|O(1)|
|depth(p)|O(d_p+1)|
|height|O(n)|
|add_root, add_left, add_right, replace, delete, attach|O(1)|


## Array-based representation of a Binary tree

An alternative representation of binary tree T is based on the way numbering the positions of T. For every position p of T, let f(p) be the integer defined as follow.
    * If p is root of T, f(p) = 0
    * If p is the left of position q then f(p) = 2f(q) + 1
    * IF p is the right of position q then f(p) = 2f(q) + 2

The advantage of array-based representation of binary tree is that position p can be represented by a single integer f(p) and the position of parent, left, right, root can be implemented using simple arthmetic operation.
    * root f(root) = 0
    * f(left of p ) = 2f(p) + 1
    * f(right of p) = 2f(p) + 2
    * f(parent of p) = round((f(p) - 1)/2)

The space used in this case is depends greatly on the shape of tree. For worst-case, tree length n need array length N=2^n-1. But in <b>HEAP</b> data structure has N = n, very efficient. Another drawback of an array representation is some update operation cannot be efficient supported. Such as the delete operation and promote its child take O(n) time because it is not just the child that move location in array but all descendants of that child.

## Tree traversal algorithms

Algorithm   preorder(T, p):

    perform the "visit" action for position p
    for each child c in T.children(p) do
        preorder(T, c)
        
Algorithm   postorder(T, p):

    for each child c in T.children(p) do
        postorder(T, c)
    perform the "visit" action for position p

Algorithm   inorder(p):
        
    for p has left child lc then
        inorder(lc)
    perform the visit action for position p
    if p has right child rc then
        inorder(rc)

Another common approach is to travel a tree so that we visit all the position at depth d then all position at depth d+1. It is suitable for application not need to visit the max depth.

Algorithm   breadfirst(T):

    initialize queue Q to contain T.root()
    while Q is not empty do
        p = Q.dequeue()
        perform visit action for position p
        for each child c in T.children(p) do
            Q.enqueue(c)

### Implementing tree traversal in python

We stated that tree T should include support for the following methods:
    * T.positions() : generate an iteration of all positions in tree T
    * iter(T) : genereat an iteration of all elements in tree T

In [ ]:
%%add_to Tree
def __iter__(self):
    for p in self.positions():
        yield p.element()


Preorder should support parameterize for position start traversal. We add a private method <i>_subtree_preorder</i> for this

In [ ]:
%%add_to Tree
def preorder(self):
    """Generate a preorder iteration of positions in tree"""
    if not self.is_empty():
        for p in self._subtree_preorder(self.root()):
            yield p

%%add_to Tree
def _subtree_preorder(self, p):
    """Generate a preorder iteration of """
    yield p
    for c in self.children(p):
        for other in self._subtree_preorder(c):
            yield other

In [ ]:
%%add_to Tree
def positions(self):
    """User preorder traversal as default order of iteration"""
    return self.preorder()

In [ ]:
%%add_to Tree
def postorder(self):
    if not self.is_empty():
        for p in self._subtree_postorder(self.root()):
            yield p
            
%%add_to Tree
def _subtree_postorder(self, p):
    for c in self.children(p):
        for other in self._subtree_postorder(c):
            yield other
    yield p

In [ ]:
from array_queue import ArrayQueue

%add_to Tree
def breadfirst(self):
    if not self.is_empty():
        fringe = ArrayQueue()
        fringe.enqueue(self.root())
        while not fringe.is_empty():
            p = fringe.dequeue()
            yield p
            for c in self.children(p):
                fringe.enqueue(c)

In [ ]:
%%add_to BinaryTree
def inorder(self):
    if not self.is_empty():
        for p in self._subtree_inorder(self.root()):
            yield p
            
def _subtree_inorder(self, p):
    if self.left(p) is not None:
        for other in self._subtree_inorder(self.left(p)):
            yield other
    yield p
    if self.right(p) is not None:
        for other in self._subtree_inorder(self.right(p)):
            yield other

def positions(self):
    return self.inorder()

## Exercise:  Table of contents